In [40]:
protein = "pdl1"

In [41]:
from Bio.PDB import PDBParser, standard_aa_names
import numpy

def aa_residues(chain):
        aa_only = []
        for i in chain:
            if i.get_resname() in standard_aa_names:
                aa_only.append(i)
        return aa_only

def calc_residue_dist(residue_one, residue_two) :
    """Returns the D between two residues"""
    min_dist = 1000
    for atom_one in residue_one:
        for atom_two in residue_two:
            diff_vector = atom_one.coord - atom_two.coord
            dist = numpy.sqrt(numpy.sum(diff_vector * diff_vector))
            if dist < min_dist:
                min_dist = dist
        # print(atom_one)
    # diff_vector  = residue_one["CA"].coord - residue_two["CA"].coord
    return min_dist

def calc_dist_matrix(chain_one, chain_two) :
    """Returns a matrix of C-alpha distances between two chains"""
    print(len(chain_one), len(chain_two))
    answer = numpy.zeros((len(chain_one), len(chain_two)), float)
    for row, residue_one in enumerate(chain_one) :
        for col, residue_two in enumerate(chain_two) :
            answer[row, col] = calc_residue_dist(residue_one, residue_two)
    return answer

In [42]:
parser = PDBParser()
structure = parser.get_structure(protein, f"../../datasets/validation/{protein}/{protein}.pdb")
model = structure[0]

light_chain = model["A"]
heavy_chain = model["B"]
ligand = model["C"]

dist_matrix_heavy = calc_dist_matrix(heavy_chain, ligand)
contact_map_heavy = dist_matrix_heavy <= 5.0

dist_matrix_light = calc_dist_matrix(light_chain, ligand)
contact_map_light = dist_matrix_light <= 5.0

# print("Contact map", contact_map)
# print("Dist matrix", dist_matrix)

print("Minimum distance", numpy.min(dist_matrix_heavy))
print("Maximum distance", numpy.max(dist_matrix_heavy))

# import pylab
# pylab.matshow(numpy.transpose(dist_matrix_heavy))
# pylab.colorbar()
# pylab.show()

# pylab.autumn()
# pylab.imshow(numpy.transpose(contact_map_heavy))
# pylab.show()

# contact_residues = [residue.get_resname() for i, residue in enumerate(heavy_chain) if numpy.any(contact_map[i, :])]
# print("Contact residues in light chain:", contact_residues)

118 113
110 113
Minimum distance 2.485146999359131
Maximum distance 62.35737228393555


In [43]:
import re
from Bio.PDB import Polypeptide
import pickle

def strip_tags(sequence):
    sequence = re.sub("<[^>]+>", "", sequence)
    return sequence.replace(" ", "")

def construct_CDR_map(sequence, cdrs ):
    # Get CDRs
    cdrs = cdrs[cdrs.find(">")+1:]
    cdr_1 = cdrs[:cdrs.find("<")]
    cdrs = cdrs[cdrs.find(">")+1:]
    cdr_2 = cdrs[:cdrs.find("<")]
    cdrs = cdrs[cdrs.find(">")+1:]
    cdr_3 = cdrs[:cdrs.find("<")]

    index_crd_1 = sequence.find("<H-CDR-1>")
    cdr_1_len = len(cdr_1)
    index_crd_2 = sequence.find("<H-CDR-2>")
    cdr_2_len = len(cdr_2)
    index_crd_3 = sequence.find("<H-CDR-3>")
    cdr_3_len = len(cdr_3)

    cdr_dict = {index_crd_1: cdr_1_len, index_crd_2: cdr_2_len, index_crd_3: cdr_3_len}

    # Replace tags in sequence with CDRs
    sequence = sequence.replace("<H-CDR-1>", "1"*len(cdr_1)).replace("<H-CDR-2>", "1"*len(cdr_2)).replace("<H-CDR-3>", "1"*len(cdr_3))
    sequence = strip_tags(sequence)
    print(sequence)
    sequence = re.sub("[^0-9]", "0", sequence)

    contact_map_list = [char for char in sequence]
    for i in range(len(contact_map_list)):
        if contact_map_list[i] == "1":
            contact_map_list[i] = True
        else:
            contact_map_list[i] = False
    
    return contact_map_list, cdr_dict

def get_cdr_offset(index, cdr_map):
    res = 0
    contigs = 0
    was_last_true = False
    for i in range(index):
        if cdr_map[i]:
            res += 1
            if not was_last_true:
                contigs += 1
            was_last_true = True
        else:
            was_last_true = False
    print(res, contigs)
    return res - contigs

In [44]:
input_sequence_file = f"../../datasets/doggy_data/{protein}_input"
output_sequence_file = f"../../datasets/doggy_data/{protein}_output"

fixed_residue_file = f"../../datasets/doggy_data/{protein}_fixed_residues"

with open(input_sequence_file, "r") as f:
    f.readline()
    input_light_sequence = f.readline()
    input_heavy_sequence = f.readline()
with open(output_sequence_file, "r") as f:
    f.readline()
    output_light_sequence = f.readline()
    output_heavy_sequence = f.readline()

# Get the CDR map
cdr_map_light, cdr_dict_light = construct_CDR_map(output_light_sequence, input_light_sequence)
cdr_map_heavy, cdr_dict_heavy = construct_CDR_map(output_heavy_sequence, input_heavy_sequence)

# Go over the full sequence
# For each residue, check if it is in contact with the ligand (distance <= 5.0) AND if it is in the CDR region
fixed_dicts = []
with open(fixed_residue_file, "wb") as f:
    light_fixed_dict = {}
    for i, residue in enumerate(light_chain):
        if numpy.any(contact_map_light[i, :]) and not cdr_map_light[i]:
            print("Light Chain")
            print(i)
            print(Polypeptide.three_to_one(residue.get_resname()), "is in contact with the ligand and not in the CDR region")
            cdr_offset = get_cdr_offset(i, cdr_map_light)
            print(cdr_offset)
            light_fixed_dict[i-cdr_offset] = Polypeptide.three_to_one(residue.get_resname())
    fixed_dicts.append(light_fixed_dict)
    heavy_fixed_dict = {}
    for i, residue in enumerate(heavy_chain):
        if numpy.any(contact_map_heavy[i, :]) and not cdr_map_heavy[i]:
            print("Heavy Chain")
            print(i)
            print(Polypeptide.three_to_one(residue.get_resname()), "is in contact with the ligand and not in the CDR region")
            cdr_offset = get_cdr_offset(i, cdr_map_heavy)
            print(cdr_offset)
            heavy_fixed_dict[i-cdr_offset] = Polypeptide.three_to_one(residue.get_resname())
    fixed_dicts.append(heavy_fixed_dict)

    print(fixed_dicts)

    pickle.dump(fixed_dicts, f)

# Generate a fixed residue input file

DIQMTQSPSSLSASVGDRVTITCRAS111111VAWYQQKPGKAPKLLIY11SFLYSGVPSRFSGSGSGTDFTLTISSLQPEDFATYYC111111111FGQGTKVEIKRTV

EVQLVESGGGLVQPGGSLRLSCAAS11111111IHWVRQAPGKGLEWVAW11111111YYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYC11111111111WGQGTLVTVSS

Heavy Chain
49
W is in contact with the ligand and not in the CDR region
8 1
7
Heavy Chain
58
Y is in contact with the ligand and not in the CDR region
16 2
14
Heavy Chain
73
T is in contact with the ligand and not in the CDR region
16 2
14
[{}, {42: 'W', 44: 'Y', 59: 'T'}]
